`text2vec-contextionary` is available throguh **Weaviate open-source ONLY**. Meaning, you will need to configure a `docker-compose.yaml` file. 

Check out [Weaviate's configuration tool](https://weaviate.io/developers/weaviate/installation/docker-compose#configurator) to create one.

## Schema

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8080") # connect to your Weaviate instance

# resetting the schema. CAUTION: THIS WILL DELETE YOUR DATA 
client.schema.delete_all()

schema = {
   "classes": [
       {
           "class": "JeopardyQuestion",
           "description": "List of jeopardy questions",
           "moduleConfig": { # configure the vectorizer
               "text2vec-contextionary": { 
                    "vectorizeClassName": "false"
                }
           },
           "properties": [
               {
                   "name": "Category",
                   "dataType": ["text"],
                   "description": "Category of the question",
               },
               {
                "name": "Question",
                "dataType": ["text"],
                "description": "The question",
               },
               {
                   "name": "Answer",
                   "dataType": ["text"],
                   "description": "The answer",
                }
            ]
        }
    ]
}

client.schema.create(schema)

print("Successfully created the schema.")

## Import the Data

In [ ]:
import requests
import json

url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

if client.is_ready():

# Configure a batch process
  with client.batch as batch:
      batch.batch_size=100
      # Batch import all Questions
      for i, d in enumerate(data):
          print(f"importing question: {i+1}")

          properties = {
              "answer": d["Answer"],
              "question": d["Question"],
              "category": d["Category"],
          }

          client.batch.add_data_object(properties, "JeopardyQuestion")
else:
  print("The Weaviate cluster is not connected.")

## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [nearText](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#neartext)

2. [nearObject](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#nearobject)

3. [nearVector](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#nearvector)

### nearText Example

Find a `JeopardyQuestion` about "animals in movies". Limit it to only 2 responses and report the distance.

In [ ]:
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_near_text({
        "concepts": ["question about animals"]
    })
    .with_limit(2) # limit the output to only 2
    .with_additional(["distance", "id"]).do() # output the distance from the query vector to the retrieved objects, along with the objects ID
)

print(json.dumps(response, indent=2))

### nearObject Example

Search through the `JeopardyQuestion` class to find the top 2 objects closest to id `5e99ed1d-aef8-41b2-a55b-105810e41560`. (The id was taken from the query above)

In [ ]:
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_near_object({
        "id": "5e99ed1d-aef8-41b2-a55b-105810e41560"
    })
    .with_limit(2) # limit the output to only 2
    .with_additional(["distance"]) # output the distance from the query vector to the retrieved objects
    .do()
)

print(json.dumps(response, indent=2))

### nearVector Example

Search through the `JeopardyQuestion` class to find the top 2 objects closest to the query vector `[-0.0125526935, -0.021168863, ... ]`

In [ ]:
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_near_vector({
        "vector": [-0.0125526935, -0.021168863, ... ] # replace with your query vector
    })
    .with_limit(2) # limit the output to only 2
    .with_additional(["distance"]) # output the distance from the query vector to the retrieved objects
    .do()
)

print(json.dumps(response, indent=2))